# Profitable App Profiles for the App Store and Google Play Markets

![Image](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTZMG2LTP5xD1lZ9SxeiWuql2IJdUG9ct4Ukg&usqp=CAU)

Table of Contents
=================

   * [1. Project Overview](#chapter1)
   * [2. Getting Started](#chapter2)
       * [2.1 Importing relevant libraries and checking their versions](#chapter2.1)
       * [2.2 Reading in datasets as lists of lists](#chapter2.2)
       * [2.3 Sneak peek at the datasets](#chapter2.3)
       
       
   * [3. Data Cleaning](#chapter3)
       * [3.1 Removing inaccurate data](#chapter3.1)
       * [3.2 Removing duplicate entries](#chapter3.2)
       * [3.3 Removing non-English apps](#chapter3.3)
       * [3.3 Isolating the free apps](#chapter3.4)
       
     
   * [4. Data Analysis](#chapter4)
       * [3.1 Removing inaccurate data](#chapter3.1)
       * [3.2 Removing duplicate entries](#chapter3.2)
       * [3.3 Removing non-English apps](#chapter3.3)
       * [3.3 Isolating the free apps](#chapter3.4)
       
   * [4. Conclusion](#-4.-Conclusion))

## 1. Project Overview <a class="anchor" id="chapter1"></a>

This is a data cleaning and analysis project. *We did not use Numpy or Pandas libraries here for a purpose.*

For this project, we'll pretend we're working as data analysts for a company that builds Android and iOS mobile apps. We make our apps available on Google Play and in the App Store.

We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that the number of users of our apps determines our revenue for any given app — the more users who see and engage with the ads, the better. Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.



## 2. Getting started <a class="anchor" id="chapter2"></a>

### 2.1 Importing relevant libraries and checking their versions <a class="anchor" id="chapter2.1"></a>

In [1]:
# Importing libraries
import requests
from csv import reader

In [2]:
# Printing verions of Python modules and packages with watermark - the IPython magic extension.
%load_ext watermark

%watermark -v -p requests,csv

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

requests: 2.26.0
csv     : 1.0



### 2.2 Reading in datasets as lists of lists <a class="anchor" id="chapter2.2"></a>

As of 1st quarter of 2021, there were approximately 2 million iOS apps available on the App Store, and 3.5 million Android apps on Google Play[<sup>1</sup>](#fn1).

Collecting data for over 4 million apps requires a significant amount of time and money, so we'll try to analyze a sample of the data instead. To avoid spending resources on collecting new data ourselves, we should first try to see if we can find any relevant existing data at no cost. Luckily, here are two data sets that seem suitable for our goals:

* [A dataset](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* [A dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

Now, we will define a function `open_data` to import the two datasets we mentioned above and save them as *list of lists*.

In [3]:
# Saving links as variables
url_android = 'https://dq-content.s3.amazonaws.com/350/googleplaystore.csv'
url_ios = 'https://dq-content.s3.amazonaws.com/350/AppleStore.csv'

# Defining the `open_data` function
def open_data(filename):
    """
    This function takes in a link to a csv file as input and 
    returns dataset in the list of lists format as output
    """
    with requests.Session() as s:
        response = s.get(filename)
        decoded_content = response.content.decode('utf-8')
        read_file = reader(decoded_content.splitlines(), delimiter=',')
        return list(read_file)
    

# Loading both datasets
android = open_data(url_android)   
android_header = android[0]
android = android[1:]

ios = open_data(url_ios) 
ios_header = ios[0]
ios = ios[1:]

### 2.3 Sneak peek at the datasets <a class="anchor" id="chapter2.3"></a>

Let's print the column names and take a look at the data dictionaries to try to identify the columns that could help us with our analysis.

In [4]:
# Exploring the columns names for the Google Play dataset
print(android_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


                                           Data Dictionary for the Google Play dataset
---------------------------------------------
| Column Name | Description | 
| ------------ | ------------- | 
| App | Application name | 
| Category | Category the app belongs to | 
| Rating | Overall user rating of the app (as when scraped) | 
| Reviews | Number of user reviews for the app (as when scraped) | 
| Size | Size of the app (as when scraped) |
| Installs | Number of user downloads/installs for the app (as when scraped)  | 
| Type | Paid or Free |
| Price | Price of the app (as when scraped) | 
| Content Rating | Age group the app is targeted at - Children / Mature 21+ / Adult |
| Genres | An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to Music, Game, Family genres. |
| Last Updated | Date when the app was last updated on Play Store (as when scraped) |
| Current Ver | Current version of the app available on Play Store (as when scraped) |
| Android Ver | Min required Android version (as when scraped) |  


In [5]:
# Exploring the columns names for the App Store dataset
print(ios_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


                                          Data Dictionary for the App Store dataset
---------------------------------------------
| Column Name | Description | 
| ------------ | ------------- | 
| id | App ID | 
| track_name | App Name | 
| size_bytes | Size (in Bytes) | 
| currency | Currency Type | 
| price | Price amount |
| rating_count_tot | User Rating counts (for all version) | 
| rating_count_ver | User Rating counts (for current version) |
| user_rating | Average User Rating value (for all version) | 
| user_rating_ver | Average User Rating value (for current version) |
| ver | Latest version code |
| cont_rating | Content Rating |
| prime_genre | Primary Genre |
| sup_devices.num | Number of supporting devices |  
| ipadSc_urls.num | Number of screenshots showed for display |  
| lang.num | Number of supported languages |  
| vpp_lic | Vpp Device Based Licensing Enabled |  

At a quick glance, the columns that might be useful for the purpose of our analysis are: `App` and `track_name`,  `Category` and 'prime_genre',  `Installs` and `rating_count_tot`, `Price` and `price`.

To make it easier to explore the two datasets, we'll write a function named `explore_data()` that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any dataset.

In [6]:
# Defining the `explore_data` function
def explore_data(dataset, start, end, rows_and_columns=False):
    """
    This function loops through the slice of a dataset, 
    and for each iteration, prints a row and adds a new line after that row, 
    and prints the number of rows and columns if `rows_and_columns` is `True`
    """
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [7]:
# Exploring the first three rows of each dataset
print('Google Play Data')
print('\n')
explore_data(android, 0, 3, True)
print('\n')
print('Apple Store Data')
print('\n')
explore_data(ios, 0, 3, True)

Google Play Data


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


Apple Store Data


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805

## 3. Data Cleaning <a class="anchor" id="chapter3"></a>

Before beginning our analysis, we need to make sure the data we analyze is accurate, or the results of our analysis will be wrong. This means that we need to do the following:

* Detect inaccurate data, and correct or remove it.
* Detect duplicate data, and remove the duplicates.

Recall that at our company, we only build apps that are free to download and install, and we design them for an English-speaking audience. This means that we'll need to do the following:

* Remove non-English apps like 爱奇艺PPS -《欢乐颂2》电视剧热播.
* Remove apps that aren't free.

Let's begin by detecting and deleting wrong data.

### 3.1 Removing inaccurate data <a class="anchor" id="chapter3.1"></a>

Let's check our datasets one by one to see if all of the rows in them have the same lengths. If some of the values are missing in a row, it will cause other values to shift left to the wrong column, which would be a problem. Of course, if the missing value is in the last column, it would be less problematic, but let's see what we have.

We'll write a function named `check_lengths()` that we can repeatedly use to print out the rows whose length is not equal to a dataset's header row and the index for each of these rows.

In [8]:
# Defining the `check_lengths` function
def check_lengths(dataset, dataset_header):
    """
    This function prints out the rows whose length is not equal 
    to a dataset's header row and the index for each of these rows
    """
    is_found = False
    for row in dataset:
        header_length = len(dataset_header)
        row_length = len(row)
        if row_length != header_length:
            print(row)
            print(dataset.index(row))
            is_found = True
    if not is_found:
        print("All Good!")

In [9]:
# Checking the row lenghts in the Google Play dataset
print(android_header)
print('\n')
check_lengths(android, android_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10472


The output above shows that row *10472* is missing the `Category` column; therefore, all other values have been shifted left to the wrong column. 
Now, let's check the second dataset:

In [10]:
# Checking the row lenghts in the Apple Store dataset
print(ios_header)
print('\n')
check_lengths(ios, ios_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


All Good!


The Apple Store dataset does not have rows of a wrong lenghts.

As the row *10472* misses the `Category` value, we'll remove this row. We put `if` clause so we can run this code more than once without deleting other rows. 

In [11]:
# Deliting inaccurate row
if android[10472][0]=='Life Made WI-Fi Touchscreen Photo Frame':
    del android[10472]  
print('New number of rows:',len(android))
print('Number of columns:', len(android[0]))

New number of rows: 10840
Number of columns: 13


### 3.2 Removing duplicate entries <a class="anchor" id="chapter3.2"></a>

The Google Play dataset has a dedicated [discussion](https://www.kaggle.com/lava18/google-play-store-apps/discussion) section, and we can see that [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/67894) raises a problem of some apps have duplicate entries.

Let's write some code and confirm the data has duplicate entries.

In [12]:
# Writing a loop to spot duplicate entries
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps in the Google Play dataset:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps in the Google Play dataset:', duplicate_apps[530:550])

Number of duplicate apps in the Google Play dataset: 1181


Examples of duplicate apps in the Google Play dataset: ['Free Blood Pressure', 'All Mental disorders', 'Nurse Grid', 'JH Blood Pressure Monitor', 'Blood Pressure', 'RT 516 VET', 'Anthem Anywhere', 'Sway Medical', "fred's Pharmacy", 'Breastfeeding Tracker Baby Log', 'Banfield Pet Health Tracker', '1800 Contacts - Lens Store', 'TextNow - free text + calls', 'Tumblr', 'Snapchat', 'Instagram', 'Periscope - Live Video', 'Snapchat', 'Instagram', 'Pinterest']


For the Google Play dataset, we have *1181* cases where an app occurs more than once. We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. To come up with a method of removing the duplicate rows, let's print the duplicate entries for a popular app and see which version is the most up-to-date.

In [13]:
# Taking a closer look at duplicate entries for the Instagram app

print('\n',android_header,'\n') 

for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)


 ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


If you examine the rows we printed for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show the data was collected at different times.

We can use this information to build a criterion for removing the duplicates. The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.

To remove the duplicates, we will:

* Create a dictionary, where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.
* Use the information stored in the dictionary and create a new data set, which will have only one entry per app (and for each app, we'll only select the entry with the highest number of reviews).

In [14]:
# Creating a dictionary with all unique apps and their highest number of reviews 

reviews_max={}
for row in android:
    name=row[0]
    n_reviews=float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name]= n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

In a previous code cell, we found that there are *1181* cases where an app occurs more than once, so the length of our dictionary (of unique apps) should be equal to the difference between the length of our data set and *1181*.

In [15]:
print('Expected length:', len(android) - 1181)
print('The length of the dictionary:', len(reviews_max))
print('Actual length:', len(android))

Expected length: 9659
The length of the dictionary: 9659
Actual length: 10840


Let's use the dictionary `reviews_max` we created above to remove the duplicate rows.
In the code cell below:

* We start by initializing two empty lists, `android_clean` and `already_added`.
* We loop through the Google Play data set, and for every iteration:
    * We isolate the name of the app and the number of reviews.
    * We add the current row (`row`) to the `android_clean`, and the app name (`name`) to the `already_added` list if:
        + The number of reviews of the current app matches the number of reviews of that app as described in the `reviews_max` dictionary; 
        + And the name of the app is not already in the `already_added` list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for `reviews_max[name] == n_reviews`, we'll still end up with duplicate entries for some apps.

In [16]:
# Removing the duplicate rows

android_clean=[]
already_added=[]
for row in android:
    name=row[0]
    n_reviews=float(row[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(row)
        already_added.append(name) #his helps us to keep track of apps that we already added
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


In [17]:
print('Expected length:', len(android) - 1181)
print('Actual length:', len(android_clean))

Expected length: 9659
Actual length: 9659


Let's take a look if there are any duplicates in the Apple Store data:

In [18]:
# Writing a loop to spot duplicate entries
duplicate_apps = []
unique_apps = []

for app in ios:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps in the Apple Store dataset:', len(duplicate_apps))

Number of duplicate apps in the Apple Store dataset: 0


For the App Store data because there are no duplicates.

### 3.3 Removing non-English apps <a class="anchor" id="chapter3.3"></a> 

For this particular analysis, we are interested only in English-speaking audience. However, if we explore the data long enough, we'll find that both data sets have apps with names that suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [33]:
print(ios[813][1])
print(ios[6731][1])
print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these apps, so we'll remove them. One way to do this is to remove each app with a name containing a symbol that isn't commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).

Behind the scenes, each character we use in a string has a corresponding number associated with it. For instance, the corresponding number for character `'a'` is 97, character `'A'` is 65, and character `'爱'` is 29,233. We can get the corresponding number of each character using the [`ord()` built-in function](https://docs.python.org/3/library/functions.html#ord).

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the [ASCII](https://en.wikipedia.org/wiki/ASCII) (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

In [25]:
# Defing a function that detects whether a character belongs to the set of common English characters or not
def eng_apps(string):
    for each in string:
        if ord(each)>127:
            return False
    return True
print(eng_apps('Instagram'))
print(eng_apps('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function seems to work fine, but some English app names use emojis or other symbols (™, etc.) that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form.

In [32]:
print(eng_apps('Docs To Go™ Free Office Suite'))
print(eng_apps('Instachat 😜'))
print(ord('™'))
print(ord('😜'))

True
True
8482
128540


If we're going to use the function we've created, we'll lose useful data since many English apps will be incorrectly labeled as non-English. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [28]:
# Defing a function that detects whether more than three characters with corresponding numbers falling outside the ASCII range
def eng_apps(string):
    is_eng=0
    for each in string:
        if ord(each)>127:
            is_eng +=1
    if is_eng > 3:
        return False
    else:
        return True
print(eng_apps('Instachat 😜'))
print(eng_apps('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the is_english() function to filter out the non-English apps for both data sets:

In [34]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if eng_apps(name):
        android_english.append(app)
        
for app in ios:
    name = app[1]
    if eng_apps(name):
        ios_english.append(app)

print('\n',android_header,'\n')
explore_data(android_english, 0, 3, True)
print('\n',ios_header,'\n')
explore_data(ios_english, 0, 3, True)


 ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13

 ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['28488221

### 3.4 Isolating the free apps <a class="anchor" id="chapter3.4"></a> 

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [35]:
# Looping through each data set to isolate the free apps in separate lists. 
android_free=[]
ios_free=[]
for row in ios_english:
    price=row[4]
    if price == '0.0':
        ios_free.append(row)

for row in android_english:
    price=row[7]
    if price == '0':
        android_free.append(row)
        
# Checking the length of each data set to see how many apps you have remaining
print(len(android_free))
print(len(ios_free))

8864
3222


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

## 4. Data Analysis <a class="anchor" id="chapter4"></a>

As we mentioned in the [Project Overview](#chapter1), our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

## Footnotes

<span id="fn1"> 1. [Number of apps available in leading app stores as of 1st quarter 2021, Statista
](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)</span>

In [20]:
import pandas as pd
app_data = pd.read_csv('AppleStore.csv')

In [21]:
print(app_data.shape)

(7197, 16)


In [22]:
# reading in data as a url from NYC Open Data
host = 'https://dq-content.s3.amazonaws.com/350/AppleStore.csv'

# saving data as a pandas dataframe named 'building_footprints_csv'
hate_Crimes = pd.read_csv(host)
print(hate_Crimes.shape)

(7197, 16)


In [23]:
app_data

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1
2,529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1
3,420009108,Temple Run,65921024,USD,0.0,1724546,3842,4.5,4.0,1.6.2,9+,Games,40,5,1,1
4,284035177,Pandora - Music & Radio,130242560,USD,0.0,1126879,3594,4.0,4.5,8.4.1,12+,Music,37,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7192,1170406182,Shark Boom - Challenge Friends with your Pet,245415936,USD,0.0,0,0,0.0,0.0,1.0.9,4+,Games,38,5,1,1
7193,1069830936,【謎解き】ヤミすぎ彼女からのメッセージ,16808960,USD,0.0,0,0,0.0,0.0,1.2,9+,Book,38,0,1,1
7194,1070052833,Go!Go!Cat!,91468800,USD,0.0,0,0,0.0,0.0,1.1.2,12+,Games,37,2,2,1
7195,1081295232,Suppin Detective: Expose their true visage!,83026944,USD,0.0,0,0,0.0,0.0,1.0.3,12+,Entertainment,40,0,1,1
